<a href="https://colab.research.google.com/github/heesukjang/Income_Prediction/blob/main/207_CNN_fromKesha_10_1_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2

#colab imports
from google.colab import drive
from google.colab.patches import cv2_imshow

#tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model, to_categorical, Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
image_directory = '/content/gdrive/MyDrive/207_final_project/idc_dataset'

In [ ]:
all_path = []
idc_image_path = []
idc_image_label = []

In [ ]:
for dir, subdir, files in os.walk(image_directory):
  path = dir + "/"
  all_path.append(path)

In [ ]:
len(all_path)

346

In [ ]:
all_path[1:10]

In [ ]:
for i in range(len(all_path)):
  for file in os.listdir(all_path[i]):
    test = file
    path = all_path[i] + test
    if path.lower().endswith('.png'):
      idc_image_path.append(path)

for i in range(len(idc_image_path)):
  split_test = idc_image_path[i]
  split_path = split_test.split("/")
  idc_image_label.append('class_' + split_path[7])

In [ ]:
#checking the length of the lists to ensure that they are equal
len(idc_image_path), len(idc_image_label)

(112716, 112716)

In [ ]:
#creating the dataframes that we will be passing to our generators
idc_data_cleaned = {'path': idc_image_path,
          'label': idc_image_label}

idc_df = pd.DataFrame(idc_data_cleaned)
# df = idc_df.sample(frac = 1)
print(idc_df)

csv_path = '/content/gdrive/MyDrive/207_final_project/'
idc_df.to_csv(csv_path + 'idc_dataframe.csv')

                                                     path    label
0       /content/gdrive/MyDrive/207_final_project/idc_...  class_0
1       /content/gdrive/MyDrive/207_final_project/idc_...  class_0
2       /content/gdrive/MyDrive/207_final_project/idc_...  class_0
3       /content/gdrive/MyDrive/207_final_project/idc_...  class_0
4       /content/gdrive/MyDrive/207_final_project/idc_...  class_0
...                                                   ...      ...
112711  /content/gdrive/MyDrive/207_final_project/idc_...  class_0
112712  /content/gdrive/MyDrive/207_final_project/idc_...  class_0
112713  /content/gdrive/MyDrive/207_final_project/idc_...  class_0
112714  /content/gdrive/MyDrive/207_final_project/idc_...  class_0
112715  /content/gdrive/MyDrive/207_final_project/idc_...  class_0

[112716 rows x 2 columns]


In [ ]:
idc_image_dataframe = '/content/gdrive/MyDrive/207_final_project/idc_dataframe.csv'
idc_image_generator = pd.read_csv(idc_image_dataframe)

In [ ]:
#counts the number of files in each class
idc_image_generator.set_index(['path','label']).count(level = 'label')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.count(level=1) should use df.groupby(level=1).count().
  


,Unnamed: 0
label,
class_0,79529
class_1,33187


In [ ]:
data_generator = ImageDataGenerator(
    validation_split = 0.3
)

train_data_generator = data_generator.flow_from_dataframe(
    idc_image_generator,
    directory = None,
    x_col =  'path',
    y_col =  'label',
    weight_col=None,
    featurewise_center = True,
    featurewise_std_normalization = True,
    #readjust the target size based on max size of images
    target_size=(50, 50),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    # validate_filenames=True
)

validation_data_generator = data_generator.flow_from_dataframe(
    idc_image_generator,
    directory = None,
    x_col =  'path',
    y_col =  'label',
    weight_col=None,
    featurewise_center = True,
    featurewise_std_normalization = True,
    #readjust the target size based on max size of images
    target_size=(50, 50),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    # validate_filenames=True
)

# test_data_generator = 

Found 112716 validated image filenames belonging to 2 classes.
Found 112716 validated image filenames belonging to 2 classes.


In [ ]:
img_height = 50
img_width = 50
img_channel = 1

In [ ]:
def get_doc_id_model():
  return tf.keras.Sequential([
                           keras.layers.Conv2D(input_shape = (img_height, img_width, img_channel), 
                                               filters=32, 
                                               kernel_size=(3, 3),
                                               padding='same', 
                                               activation='relu'),
                           
                           keras.layers.MaxPooling2D(pool_size=(2, 2),
                                                  strides=(2, 2)),
                           
                           keras.layers.Conv2D(input_shape = (img_height, img_width, img_channel),
                                               filters=64, 
                                               kernel_size=(3, 3), 
                                               padding='same', 
                                               activation='relu'),
                              
                           keras.layers.MaxPooling2D(pool_size=(2, 2),
                                                  strides=(2, 2)),
                          
                           keras.layers.Conv2D(input_shape = (img_height, img_width, img_channel), 
                                               filters=128, 
                                               kernel_size=(3, 3), 
                                               padding='same', 
                                               activation='relu'),
                          
                           keras.layers.MaxPooling2D(pool_size=(2, 2), 
                                                  strides=(2, 2)),
                           
                           keras.layers.Flatten(),
                           
                           keras.layers.Dense(units = 512, 
                                              activation = 'relu'),
                           
                          #  keras.layers.Dense(units = 512, 
                          #                     activation = 'relu'),
                           
                           keras.layers.Dense(units = 2, 
                                              activation = 'softmax')
])

In [ ]:
model = get_doc_id_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.001), 
                    loss=keras.losses.categorical_crossentropy, 
                    metrics=['accuracy']
              )

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 50, 50, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 25, 25, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 6, 6, 128)       

In [ ]:
val_acc_early_stopping = EarlyStopping(monitor = 'val_acc', 
                                       patience = 5, 
                                       verbose = 1,
                                       mode = 'auto')

In [ ]:
hist = model.fit(train_data_generator,
                 epochs=10,
                      #  callbacks=[val_acc_early_stopping]
                 validation_data = validation_data_generator
                 )